In [4]:
import os
import numpy as np
import pandas as pd
import pre_processing_utils
from matplotlib.image import imread
from tensorflow.keras.metrics import AUC as auc
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data_dir = 'chest_xray'

In [6]:
test_path = data_dir + '/test'
train_path = data_dir + '/train'
val_path = data_dir + '/val'

In [7]:
image_shape = (300, 300, 3)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
image_gen = ImageDataGenerator(fill_mode='nearest')

In [10]:
image_gen.flow_from_directory(pre_processing_utils.train_path)

Found 5216 images belonging to 2 classes.


In [11]:
image_gen.flow_from_directory(pre_processing_utils.test_path)

Found 6240 images belonging to 2 classes.


In [12]:
image_gen.flow_from_directory(pre_processing_utils.validation_path)

Found 16 images belonging to 2 classes.


<h2>Building a Model</h2>

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, Layer, concatenate, GlobalAveragePooling2D,Activation, Softmax
from tensorflow.keras.metrics import Accuracy, AUC, Recall, Precision

In [2]:
class fire_module(Layer):
    
    def __init__(self, squeeze_size=16, expand_size=64):
        super(fire_module, self).__init__()
        self.squeeze = Conv2D(filters=squeeze_size, kernel_size=(1,1), padding='valid', activation='relu', name= "sq1x1")
        self.exp1_1 = Conv2D(filters=expand_size, kernel_size=(1,1), padding='valid', activation='relu', name= "exp1x1")
        self.exp3_3 = Conv2D(filters=expand_size, kernel_size=(3,3), padding='same', activation='relu', name= "exp3x3")
    
    def call(self, input):
        squeezed_value = self.squeeze(input)
        exp1_1_value = self.exp1_1(squeezed_value)
        exp3_3_value = self.exp3_3(squeezed_value)
        return concatenate([exp1_1_value, exp3_3_value], axis=-1, name= 'concat')

In [3]:
model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(3,3), input_shape=image_shape, activation='relu'))

model.add(fire_module(6,8))

model.add(fire_module(12,16))

model.add(fire_module(18,24))

model.add(Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid'))
model.add(GlobalAveragePooling2D())

#model.add(Flatten())
#model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=[AUC(), Recall(), Precision()]) 

NameError: name 'image_shape' is not defined

In [ ]:
model = Sequential()


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 8)       224       
_________________________________________________________________
fire_module (fire_module)    (None, 298, 298, 16)      550       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
fire_module_1 (fire_module)  (None, 99, 99, 32)        2156      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
fire_module_2 (fire_module)  (None, 33, 33, 48)        4962      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 48)        0

<h2>Regularization</h2>

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import normalize

In [16]:
early_stop = EarlyStopping(monitor='val_auc', mode='max', patience=2)
batch_size = 64

In [17]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size = image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode = 'binary')

Found 5216 images belonging to 2 classes.


In [18]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode='binary',
                                               shuffle=False) # Don't want to shuffle test data and lose labels

Found 6240 images belonging to 2 classes.


In [19]:
test_image_gen.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

<h3>Running the Model</h3>

In [20]:
# Running without early stop for now - not enough examples for 'val_loss'?
results = model.fit(train_image_gen, epochs=20,
                             validation_data=test_image_gen,
                    callbacks=[early_stop]
                   )

Train for 82 steps, validate for 98 steps
Epoch 1/20
82/82 [==============================] - 431s 5s/step - loss: 0.9311 - auc: 0.5721 - recall: 0.9182 - precision: 0.7403 - val_loss: 0.7846 - val_auc: 0.7352 - val_recall: 1.0000 - val_precision: 0.5002
Epoch 2/20
82/82 [==============================] - 400s 5s/step - loss: 0.4407 - auc: 0.8543 - recall: 0.9773 - precision: 0.7872 - val_loss: 0.7172 - val_auc: 0.8227 - val_recall: 0.9962 - val_precision: 0.5050
Epoch 3/20
82/82 [==============================] - 397s 5s/step - loss: 0.3951 - auc: 0.9004 - recall: 0.9703 - precision: 0.8170 - val_loss: 0.7053 - val_auc: 0.8585 - val_recall: 0.9981 - val_precision: 0.5055
Epoch 4/20
82/82 [==============================] - 397s 5s/step - loss: 0.3654 - auc: 0.9383 - recall: 0.9719 - precision: 0.8365 - val_loss: 0.7695 - val_auc: 0.8615 - val_recall: 0.9987 - val_precision: 0.5020
Epoch 5/20
82/82 [==============================] - 398s 5s/step - loss: 0.3484 - auc: 0.9357 - recall: 0.

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[['accuracy', 'val_accuracy']].plot()

In [ ]:
metrics[['auc_2', 'val_auc_2']].plot()

In [ ]:
metrics[['loss', 'val_loss']].plot()

<p>We see that the model is overfitting significantly.</p>

In [17]:
model.save('image_classifier_accuracy.h5')

NotImplementedError: Layers with arguments in `__init__` must override `get_config`.